In [1]:
%load_ext autoreload
%autoreload 2

import torch

from torch.utils.data import DataLoader
from dataset import RecoveryRateDataset
import os
from training_utils import compute_mean_and_std, apply_std_scaling, set_seed
from model_lightning import LSTMEstimator
#from training_loop import model_train

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.profiler import PyTorchProfiler
from pytorch_lightning.loggers import WandbLogger

SEED = 1234
seed_everything(SEED, workers=True)

import os
os.environ["WANDB_START_METHOD"] = "thread"
import wandb

/home/cal-display1/Documents/4tb/aksoy/start_env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Global seed set to 1234


In [2]:
root_dir = os.getcwd().split('Start_PyCharm')[0]
data_file_path = root_dir + 'Start_PyCharm/state_vector_dataframes_as_training_data/data/airport_state_weather_prediction_added.pickle'


SEQUENCE_LENGTH = 3

features_to_drop = ['reg_type', 'reg_bool_type', 'reg_cause', "weather_prediction"]
rr_dataset = RecoveryRateDataset(data_file_path, features_to_drop=features_to_drop, sequence_length=SEQUENCE_LENGTH,
                                 fill_with='backfill')


train_val_test_ratios = (0.6, 0.2, 0.2)

lengths=[int(len(rr_dataset)*ratio) for ratio in train_val_test_ratios]

if sum(lengths) != len(rr_dataset):
    offset = sum(lengths) - len(rr_dataset)
    lengths[0] = lengths[0] - offset
                                                                      
train, val, test = torch.utils.data.random_split(rr_dataset, lengths=lengths)

#Compute mean and std for standard scaling.
#training_mean, training_std = compute_mean_and_std(train)

#scaled_training_data = apply_std_scaling(train, training_mean, training_std)
#scaled_val_data = apply_std_scaling(val, training_mean, training_std)
#scaled_test_data = apply_std_scaling(test, training_mean, training_std)

In [4]:
val_data_loader = DataLoader(val, batch_size=256, num_workers=4)
test_data_loader = DataLoader(test)
BATCH_SIZE = 256

In [ ]:
experiment = [16]

for _ in experiment:
    training_data_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True,
                                  num_workers=4)
    main()

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                      | Type       | Params
---------------------------------------------------------
0 | feature_extracting_layers | Sequential | 8.4 M 
1 | lstm_layer                | LSTM       | 9.5 M 
2 | estimator_layers          | Sequential | 7.9 M 
---------------------------------------------------------
25.7 M    Trainable params
0         Non-trainable params
25.7 M    Total params
102.961   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1234


Training: 0it [00:00, ?it/s]

In [16]:
def main():
    #Training parameters.
    LR = 1e-4

    
    model = LSTMEstimator(len(rr_dataset.feature_names), initial_dense_layer_size=1024, dense_parameter_multiplier=1,
                          dense_layer_count=8, lstm_layer_count=4, lstm_hidden_units=512, dropout=0.2,
                          sequence_length=SEQUENCE_LENGTH, lr=LR, loss="huber")


    #wandb_instance = wandb.init(project='recovery_rate')  
    profiler = PyTorchProfiler()
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    wandb_logger = WandbLogger(name=f"dense_8_huber", project='recovery_rate', log_model="all")
    checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")
    wandb_logger.experiment.config.update({'batch_size': BATCH_SIZE, 'init_lr': LR})
                                          #'mean': training_mean, 'std': training_std})
    

    lightning_trainer = Trainer(logger=wandb_logger, devices="auto", callbacks=[checkpoint_callback, lr_monitor], 
                                deterministic=True, accelerator="gpu", max_time="00:20:00:00")
    
    
    lightning_trainer.fit(model, training_data_loader, val_data_loader)
    wandb_logger.finalize("completed")
    wandb.finish(quiet=True)
    